In [15]:
import src.constants; reload(src.constants)
from src.constants import fn_frsq_venues_dc, fn_segments_dc, epsg_dc
from src.constants import fn_mapping_for_fs
import geopandas as gp
import pandas as pd

In [3]:
data_dir = 'data/'
path_frsq_venues_dc = data_dir + fn_frsq_venues_dc
path_mapping_for_fs = data_dir+fn_mapping_for_fs

frsq_venues_dc = gp.read_file(path_frsq_venues_dc)

In [57]:
def mapping_for_fs(path_mapping_for_fs):
    result = []
    with open(path_mapping_for_fs) as f:
        lvs = ['','','','','','','','','']
        for line in f:
            line = line.split('\t')
            lv = len(line)
            node = line[-1].strip()
            lvs[lv]=node
            parent = '/'.join(lvs[:lv])
            result.append([lv, node, parent, lvs[1]])
    new_taxonomy = pd.DataFrame(result,columns=['lv','tag','parents','top_parent'])
    return pd.Series(new_taxonomy.top_parent.values,index=new_taxonomy.tag).to_dict()



In [56]:
def map_frsq_venues_to_poi_category(frsq_venues_gpdf, path_mapping_for_fs, debug=False):
    print '===========mapping  frsq venues to poi categories==========='
    mapping = mapping_for_fs(path_mapping_for_fs)
    frsq_venues_gpdf['mapped'] = frsq_venues_gpdf.category.apply(lambda x: mapping[x.encode('utf-8')] if x.encode('utf-8') in mapping else 'no category')
    
    if debug:
        frsq_venues_gpdf_no_category = frsq_venues_gpdf[frsq_venues_gpdf.mapped=='no category']
        print 'venues without poi category: #venues={}, #frsq_categories={}'.format(len(frsq_venues_gpdf_no_category), len(pd.unique(frsq_venues_gpdf_no_category.category)))
        print 'top ten frsq_categories', frsq_venues_gpdf[frsq_venues_gpdf.mapped=='no category'].category.value_counts().head(10).to_dict()
    
    return frsq_venues_gpdf

frsq_venues_gpdf = map_frsq_venues_to_poi_category(frsq_venues_dc, path_mapping_for_fs, debug=True)
poi_distribution = frsq_venues_gpdf.mapped.value_counts()
poi_distribution.index.name='category'
print 'mapped poi distribution', poi_distribution.to_dict()



===========mapping  frsq venues to poi categories===========
venues without poi category: #venues=3139, #frsq_categories=63
top ten frsq_categories {u'': 1447, u'Building': 882, u'TV Station': 11, u'Exhibit': 51, u'Moving Target': 121, u'Zoo': 14, u'Festival': 6, u'Conference': 7, u'Intersection': 55, u'Road': 450}
mapped poi distribution {'professional service': 6620, 'transportation': 1927, 'cycling facilities': 165, 'food': 3937, 'residence': 1980, 'retail shop': 3753, 'nightlife spot': 1593, 'no category': 3139, 'art': 1007, 'schools&university': 1709, 'outdoors and recreation': 4357}
